<img src="../../assets/header.gif"
     width="4000" 
     height="100"/>
<hr style="color:#c6cde1;">
<p align="center">
    <b style="font-size:2.5vw; color:#c6cde1; font-weight:bold;">
    Upper-level: Power flow formulations comparison
    </b>
</p>
<hr style="color:#c6cde1;">
<b>Description</b><br>
<p align="justify">
    The purpose of this notebook is to compare the DNEP formulations from the two papers
    <ul>
        <li style="margin-bottom:10px"> Jabr, R. A. (2012). Polyhedral formulations and loop elimination constraints for distribution network expansion planning. IEEE Transactions on Power Systems, 28(2), 1888-1897. and,</li>
        <li style="margin-bottom:10px"> Franco, J. F., Rider, M. J., & Romero, R. (2014). A mixed-integer quadratically-constrained programming model for the distribution system expansion planning. International Journal of Electrical Power & Energy Systems, 62, 265-272.</li>
    </ul> 
<br>
<b>Author</b><br><br> 
<i>Manon Cornet</i>

[comment]: <> (Section)
<hr style="color:#c6cde1;">
<p align="center">
    <b style="font-size:2vw; color:#c6cde1;">
    1. Initialization
    </b>
</p>
<hr style="color:#c6cde1;">

[comment]: <> (Description)
<p align="justify">
    In this section, one will be able to:
    <ul>
        <li style="margin-bottom:10px">Initialize all the librairies needed for the project;</li>
        <li style="margin-bottom:10px">Include the codes containing the two formulations</li>
    </ul> 

In [18]:
# -- LIBRAIRIES --#

import XLSX, DataFrames
using JuMP, Gurobi, Printf, Dates
using GraphRecipes, Graphs, Plots
using Test, PyPlot
using PrettyTables
using LaTeXStrings

In [19]:
# -- MODEL FILES --

include("UL_Jabr.jl")
include("UL_BFM.jl")

UL_BFM_2P (generic function with 1 method)

In [3]:
# -- FUNCTION TO PRINT TITLE --

function print_title(title::String)
    println(title)
    for i in 1:length(title) @printf("-") end
    @printf("\n")
end

print_title (generic function with 1 method)

In [20]:
# -- FUNCTION TO PRINT 1D ARRAY --
function print_vector(vector::Vector{<:Number}, units)
    println("["*join(vector, ", ")*"] [$units]")
end

print_vector (generic function with 1 method)

In [21]:
# -- UTIL FUNCTIONS --
function sig_round(x)
    for (i, a) in enumerate(x)
        if isa(a, AbstractFloat)
            x[i] = round(a, sigdigits=4)
            if abs(a) < 1e-3
                x[i] = 0
            end
        end
    end
    return x
end

function if_small(x)
    for (i, a) in enumerate(x)
        if abs(a) < 1e-6
            x[i] = 0
        end
    end
    return x #round.(x, digits=6)
end

if_small (generic function with 1 method)

[comment]: <> (Section)
<hr style="color:#c6cde1;">
<p align="center">
    <b style="font-size:2vw; color:#c6cde1;">
    2. Definition of the network model 
    </b>
</p>
<hr style="color:#c6cde1;">

[comment]: <> (Description)
<p align="justify">
    In this section, one will be able to:
    <ul>
        <li style="margin-bottom:10px">Load a network model from an Excel file;</li>
        <li style="margin-bottom:10px">Initialize the parameters defining the network</li>
    </ul> 

In [22]:
# -- LOADING OF THE EXCEL FILE --
root_dir = normpath(joinpath(@__FILE__,"..","..",".."))
XLSX_FILE_PATH = joinpath(root_dir, "NetworkModels/network_Nahman_Peric_2S23H.xlsx")

"../../NetworkModels/network_Nahman_Peric_2S23H.xlsx"

In [23]:
# -- DEFINITION OF THE PER UNIT BASIS --
# Values obtained by reading the Paper "A Constructive Heuristic Algorithm for Distribution System Planning"

const BASE_VOLTAGE    = 34.5                             # [kV]
const BASE_POWER      = 1                                # [MVA]
const BASE_CURRENT    = BASE_POWER / BASE_VOLTAGE        # [kA]
const BASE_ADMITTANCE = BASE_CURRENT / BASE_VOLTAGE      # [S]
const BASE_IMPEDANCE  = 1/BASE_ADMITTANCE                # [Ohm]
print_title("Per-unit basis:")

@printf("Base Admittance : %.3e [S]\n", BASE_ADMITTANCE)
@printf("Base Impedance  : %.3e [Ohm]\n", BASE_IMPEDANCE)
@printf("Base Voltage    : %.3e [kV]\n", BASE_VOLTAGE)
@printf("Base Current    : %.3e [kA]\n", BASE_CURRENT)
@printf("Base Power      : %.3e [MVA]\n", BASE_POWER)

Per-unit basis:
---------------
Base Admittance : 8.402e-04 [S]
Base Impedance  : 1.190e+03 [Ohm]
Base Voltage    : 3.450e+01 [kV]
Base Current    : 2.899e-02 [kA]
Base Power      : 1.000e+00 [MVA]


In [24]:
# -- FUNCTION THAT PROCESSES THE LINE PROPRETIES  --

function process_conductors(df_cond::DataFrames.DataFrame, 
                            len_lines::Vector{Float64},  
                            nb_lines::Integer, 
                        )

    nb_cond = DataFrames.nrow(df_cond)
    max_i   = Array{Float64}(undef, nb_lines, nb_cond) # absolute, [pu]
    r       = Array{Float64}(undef, nb_lines, nb_cond) # absolute, [pu]
    x       = Array{Float64}(undef, nb_lines, nb_cond) # absolute, [pu]
    g       = Array{Float64}(undef, nb_lines, nb_cond) # absolute, [pu]
    b       = Array{Float64}(undef, nb_lines, nb_cond) # absolute, [pu]
    
    line_cost = Array{Float64}(undef, nb_lines, nb_cond) # [€/km]

    # Only take the first conductors of the list in the file
    for l in 1:nb_lines
        for k in 1:nb_cond
            max_i[l, k] = df_cond.max_i_ka[k] ./ BASE_CURRENT

            r[l, k] = len_lines[l] * df_cond.r_ohm_per_km[k] ./ BASE_IMPEDANCE
            x[l, k] = len_lines[l] * df_cond.x_ohm_per_km[k] ./ BASE_IMPEDANCE
            y = 1/(r[l, k]+im*x[l, k])

            g[l, k] = real(y) 
            b[l, k] = abs(imag(y))

            line_cost[l, k] = df_cond.cost_kdollars_per_km[k] * len_lines[l]
        end
    end
    return max_i, line_cost, r, x, g, b
end

process_conductors (generic function with 1 method)

In [25]:
# -- FETCH THE DATA FROM THE EXCEL SHEET --

df_bus  = DataFrames.DataFrame(XLSX.readtable(XLSX_FILE_PATH, "bus"))
df_line = DataFrames.DataFrame(XLSX.readtable(XLSX_FILE_PATH, "line"))
df_cond = DataFrames.DataFrame(XLSX.readtable(XLSX_FILE_PATH, "conductor"))

Row,idx,name,d_mm,q_mm2,r_ohm_per_km,x_ohm_per_km,max_i_ka,cost_kdollars_per_km
,Any,Any,Any,Any,Any,Any,Any,Any
1,1,Poppy,9.35,53.5,0.5502,0.429,0.23,10
2,2,Oxlip,13.26,107.3,0.2747,0.402,0.34,12
3,3,Daisy,14.88,135.3,0.218,0.394,0.46,15
4,4,Tulip,16.92,170.6,0.1732,0.381,0.53,20


In [26]:
# -- LINE PARAMETERS DEFINITION --

L_size = DataFrames.nrow(df_line)                                   # Number of lines in the network
L      = 1:L_size                                                   # Line set
line_ends = [(df_line.from_bus[l], df_line.to_bus[l]) for l in L]   # Indices of the line extremities
len_lines = convert(Vector{Float64}, df_line.length_km)             # Line lengths [km]

print_title("Network lines summary:")
@printf("Number of lines : %d\n", L_size)

Network lines summary:
----------------------
Number of lines : 34


In [27]:
# -- DEFINITION OF THE PHYSICAL QUANTITIES ASSOCIATED TO NETWORK LINES --

K_size = DataFrames.nrow(df_cond)   # Number of conductor types
K      = 1:K_size                   # Set of conductors

max_i, line_cost, R, X, G, B, = process_conductors(df_cond, len_lines, L_size)

print_title("Conductor properties:")
println(df_cond)

Conductor properties:
---------------------
4×8 DataFrame
 Row │ idx  name   d_mm   q_mm2  r_ohm_per_km  x_ohm_per_km  max_i_ka  cost_kdollars_per_km 
     │ Any  Any    Any    Any    Any           Any           Any       Any                  
─────┼──────────────────────────────────────────────────────────────────────────────────────
   1 │ 1    Poppy  9.35   53.5   0.5502        0.429         0.23      10
   2 │ 2    Oxlip  13.26  107.3  0.2747        0.402         0.34      12
   3 │ 3    Daisy  14.88  135.3  0.218         0.394         0.46      15
   4 │ 4    Tulip  16.92  170.6  0.1732        0.381         0.53      20


In [28]:
# -- BUS PARAMETERS DEFINITION --

N_size = DataFrames.nrow(df_bus)            # Number of buses in the network
N      = 1:N_size                           # Buses set

Ns_size = sum(df_bus.type .== "substation") # Number of substation buses
Nu_size = sum(df_bus.type .== "user")       # Number of load nodes

Ns = 1:Ns_size                              # Set of substation buses
Nu = (1:Nu_size) .+ Ns_size                 # Set of load buses

print_title("Network buses summary:")
@printf("Number of buses            : %d\n", N_size)
@printf("Number of load buses       : %d\n", Nu_size)
@printf("Number of substation buses : %d\n", Ns_size)

Network buses summary:
----------------------
Number of buses            : 23
Number of load buses       : 21
Number of substation buses : 2


In [29]:
# -- DEFINITION OF THE PHYSICAL QUANTITIES ASSOCIATED TO NETWORK BUSES --

# Limits on voltage
MIN_VOLTAGE = 0.95  # [pu]
MAX_VOLTAGE = 1.05  # [pu]

# Demand at buses
# Assumption: load power factor is constant for all loads and is lagging (inductive)
cos_phi = 0.95
S_D     = convert(Vector{Float64}, df_bus.S_D_mva) ./ BASE_POWER
peak_demand = sum(S_D)
P_D     = S_D * cos_phi
Q_D     = S_D * sin(acos(cos_phi))

x_coordinates = convert(Vector{Float64}, df_bus.x)
y_coordinates = convert(Vector{Float64}, df_bus.y)

print_title("Network buses additional info:")
@printf("Power factor  : %.3g\n", cos_phi)
@printf("Peak demand   : %.3g\n", peak_demand)
@printf("Voltage range : [%1.3g, %1.3g] [pu]", MIN_VOLTAGE, MAX_VOLTAGE)

Network buses additional info:
------------------------------
Power factor  : 0.95
Peak demand   : 7.04
Voltage range : [0.95, 1.05] [pu]

In [30]:
# -- SUBSTATION PARAMETERS DEFINITION --

S_rating_init    = convert(Vector{Float64}, df_bus.S_G_init_mva[Ns]) ./ BASE_POWER # [pu]
S_rating_max     = convert(Vector{Float64}, df_bus.S_G_max_mva[Ns]) ./ BASE_POWER # [pu]
sub_install_cost = 1e3      # k$
sub_op_cost      = 0.1*1e-3 # k$/kVah^2

print_title("Substation buses additional info:")
print("Initial rating of substations: "); print_vector(S_rating_init, "pu");
print("Maximum rating of substations: "); print_vector(S_rating_max, "pu");
@printf("Substation construction cost : %.3g [k\$]\n", sub_install_cost)
@printf("Substation operation cost    : %.3g [k\$/kVA^2]", sub_op_cost)

Substation buses additional info:
---------------------------------
Initial rating of substations: [0.0, 0.0] [pu]
Maximum rating of substations: [15.0, 15.0] [pu]
Substation construction cost : 1e+03 [k$]
Substation operation cost    : 0.0001 [k$/kVA^2]

In [31]:
# -- LINK BTW LINES AND NODES --

Omega_sending   = Dict(n => [] for n in N)
Omega_receiving = Dict(n => [] for n in N)
for l in L
    push!(Omega_sending[line_ends[l][1]], l)
    push!(Omega_receiving[line_ends[l][2]], l)
end

print_title("Link btw lines and nodes:")
println("Omega sending  : $Omega_sending")
println("Omega receiving: $Omega_receiving")

Link btw lines and nodes:
-------------------------
Omega sending  : Dict{Int64, Vector{Any}}(5 => [21, 22], 16 => [], 20 => [], 12 => [18], 8 => [28, 29, 30, 33], 17 => [17], 1 => [1], 19 => [5, 6, 7], 22 => [8, 9], 23 => [19, 20], 6 => [25, 26], 11 => [10, 11], 9 => [32], 14 => [24], 3 => [34], 7 => [27], 4 => [23, 31], 13 => [12], 15 => [13, 14, 15], 2 => [], 10 => [2, 3, 4], 18 => [16], 21 => [])
Omega receiving: Dict{Int64, Vector{Any}}(5 => [20], 16 => [8, 25, 34], 20 => [2, 5], 12 => [15], 8 => [27], 17 => [16], 1 => [], 19 => [4], 22 => [7], 23 => [18], 6 => [23, 24], 11 => [9], 9 => [29, 31], 14 => [3, 17, 19, 21], 3 => [32, 33], 7 => [26], 4 => [22, 30], 13 => [11], 15 => [12], 2 => [28], 10 => [1], 18 => [14], 21 => [6, 10, 13])


[comment]: <> (Section)
<hr style="color:#c6cde1;">
<p align="center">
    <b style="font-size:2vw; color:#c6cde1;">
    3. Definition of the objective function parameters 
    </b>
</p>
<hr style="color:#c6cde1;">

[comment]: <> (Description)
<p align="justify">
    In this section, one will be able to:
    <ul>
        <li style="margin-bottom:10px">Set the parameters that are required in the objective function;</li>
    </ul> 

In [32]:
# -- FUNCTION FOR COMPUTING THE NPV --

function PV_coeff(tau, lambda)
    return (1 - 1/(1 + tau)^lambda)/tau  
end

# Capital recovery rate formula
# tau: interest rate 
# n : number of annuity received
function CRF(tau, n)
    return (tau * (1 + tau)^n)/((1 + tau)^n - 1)  
end

CRF (generic function with 1 method)

In [33]:
# -- OBJECTIVE FUNCTION PARAMETERS --

nb_years_planning = 1
delta_t           = 1 # [h]

tau = 0.1

line_loss_factor = 0.35     # phi_l : loss factor of lines
sub_loss_factor  = 0.35     # phi_s : cost per energy lost [€/kWh]

K_l = CRF(tau, 1)           # Capital recovery rate of line constructions
K_s = CRF(tau, 1)           # Capital recovery rate of substation construction or reinforcement

loss_cost = 0.05*1e-3       # [k$/kWh]
tau_l     = tau             # tau_l : interest rate of circuits
tau_s     = tau             # tau_s : interest rate of substations

f_l = PV_coeff(tau_s, nb_years_planning)
f_s = PV_coeff(tau_s, nb_years_planning)

0.9090909090909094

In [34]:
# -- CREATION OF THE NETWORK DICT --
# Rmq: all the costs are in k$
network_dict = Dict(:bus       => (N, Omega_sending, Omega_receiving, MIN_VOLTAGE, MAX_VOLTAGE, x_coordinates, y_coordinates),
                    :load_bus  => (Nu, P_D, Q_D, delta_t),
                    :sub_bus   => (Ns, S_rating_init, S_rating_max),
                    :line      => (L, line_ends, max_i, R, X, G, B),
                    :conductor => (K)
                    )

Jabr_obj_fct_dict = Dict(:LF  => (line_loss_factor, sub_loss_factor),
                         :CRF => (K_l, K_s),
                         :tau => (tau_l, tau_s),
                         :costs => (loss_cost, sub_install_cost, sub_op_cost, line_cost)
                        )

BFM_obj_fct_dict = Dict(:LF  => (line_loss_factor, sub_loss_factor),
                        :CRF => (K_l, K_s),
                        :NPV_coeff => (f_l, f_s),
                        :costs => (loss_cost, sub_install_cost, sub_op_cost, line_cost)
                        )

Dict{Symbol, Tuple{Float64, Float64, Vararg{Any}}} with 4 entries:
  :NPV_coeff => (0.909091, 0.909091)
  :LF        => (0.35, 0.35)
  :CRF       => (1.1, 1.1)
  :costs     => (5.0e-5, 1000.0, 0.0001, [2.02092 2.4251 3.03137 4.04183; 6.972…

[comment]: <> (Section)
<hr style="color:#c6cde1;">
<p align="center">
    <b style="font-size:2vw; color:#c6cde1;">
    4. Test of the Jabr and branch flow models
    </b>
</p>
<hr style="color:#c6cde1;">

[comment]: <> (Description)
<p align="justify">
    In this section, one will be able to:
    <ul>
        <li style="margin-bottom:10px">Set the parameters that are required in the objective function;</li>
    </ul> 

In [35]:
# -- JABR FORMULATION --
Jabr_model, Jabr_var_values, Jabr_var_sets = UL_Jabr(network_dict, Jabr_obj_fct_dict)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Set parameter TimeLimit to value 200
Set parameter MIPGap to value 1e-06
Set parameter Presolve to value 0
Set parameter MIPGap to value 1e-06
Set parameter TimeLimit to value 200
Set parameter Presolve to value 0
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2944 rows, 4888 columns and 8737 nonzeros
Model fingerprint: 0xfa6dd54f
Model has 2 quadratic objective terms
Model has 138 quadratic constraints
Variable types: 4716 continuous, 172 integer (172 binary)
Coefficient statistics:
  Matrix range     [1e-01, 3e+09]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e-01, 1e+03]
  QObjective range [7e-01, 7e-01]
  Bounds range     [9e-01, 1e+00]
  RHS range        [1e-01, 2e+01]
         Consider reformulating model or setting NumericFocus parameter
  

(A JuMP Model
Minimization problem with:
Variables: 4344
Objective function type: QuadExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 762 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 683 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 955 constraints
`Vector{VariableRef}`-in-`MathOptInterface.SecondOrderCone`: 2 constraints
`Vector{AffExpr}`-in-`MathOptInterface.RotatedSecondOrderCone`: 136 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 2919 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 161 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 23 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 172 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: I_sqr, P_G, P_ij, P_ji, Q_G, Q_ij, Q_ji, S_G, V_sqr, X_i_ij, X_ij_im, X_ij_re, active_balance, alpha, beta, cone_28, current_limit, line_constructed, number_of

In [40]:
# -- DISTFLOW FORMULATION --
#
BFM_model, BFM_var_values, BFM_var_sets = UL_BFM_1P(network_dict, BFM_obj_fct_dict)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Set parameter TimeLimit to value 200
Set parameter Presolve to value 0
Set parameter TimeLimit to value 200
Set parameter Presolve to value 0
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 1809 rows, 944 columns and 6389 nonzeros
Model fingerprint: 0x2239f087
Model has 2 quadratic objective terms
Model has 36 quadratic constraints
Variable types: 738 continuous, 206 integer (206 binary)
Coefficient statistics:
  Matrix range     [7e-10, 3e+02]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e-06, 1e+03]
  QObjective range [6e-01, 6e-01]
  Bounds range     [9e-01, 1e+00]
  RHS range        [1e-01, 2e+01]
Variable types: 737 continuous, 207 integer (206 binary)

Root relaxation: objective 1.628113e+02, 2223 iterations, 0.01 seconds (0.01 work units)

 

(A JuMP Model
Minimization problem with:
Variables: 808
Objective function type: QuadExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 184 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 602 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 887 constraints
`Vector{VariableRef}`-in-`MathOptInterface.SecondOrderCone`: 2 constraints
`Vector{AffExpr}`-in-`MathOptInterface.RotatedSecondOrderCone`: 34 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 63 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 195 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 23 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 206 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: I_sqr, I_sqr_k, P_G, P_ij, P_ij_k, P_limit1, P_limit2, P_limit3, P_limit4, Q_G, Q_ij, Q_ij_k, Q_limit1, Q_limit2, Q_limit3, Q_limit4, S_G, V_sqr, active_balance, a

In [73]:
# -- OUTPUT FUNCTIONS --
#
function print_cost_results(model, network_dict, obj_dict; Jabr=false, latex=false)

    # ---------------- Network data ----------------
    N, _, _, _, _, _, _ = network_dict[:bus]
    Ns, _, _ = network_dict[:sub_bus]
    Nu, _, _ , _ = network_dict[:load_bus]
    L, _, _, _, _, _, _ = network_dict[:line]
    K = network_dict[:conductor] 
    base_power = BASE_POWER
    HOURS_PER_YEAR = 8760 # h/year
    delta_t = 1 # 1h

    # ---------------- Costs data ----------------
    loss_cost, sub_install_cost, sub_op_cost, line_cost = obj_dict[:costs]
    line_loss_factor, sub_loss_factor = obj_dict[:LF]
    K_l, K_s = obj_dict[:CRF]
   

    # Investements to build new lines in k€
    DSO_fixed_line = K_l * value(sum(model[:alpha][l, k] * line_cost[l, k] for l in L, k in K)) # /year
   
    # Investements to build new substations in k€
    DSO_fixed_sub  = K_s * value(sum(model[:beta][i] * sub_install_cost for i in Ns))

    # Operational costs of losses on the length of the horizon
    if Jabr
        tau_l, tau_s = obj_dict[:tau]

        DSO_loss = HOURS_PER_YEAR * (1 + tau_l) * line_loss_factor * loss_cost * delta_t * 
                    value(sum((model[:P_G][i] - P_D[i]) * BASE_POWER for i in N)) 

        DSO_sub_op_costs = HOURS_PER_YEAR * (1 + tau_s) * sub_loss_factor * value(sum(sub_op_cost
                    * (model[:P_G][i]^2 + model[:Q_G][i]^2) * (delta_t * BASE_POWER)^2 for i in Ns))
    else
        f_l, f_s = obj_dict[:NPV_coeff]
        
        DSO_loss = HOURS_PER_YEAR * f_l * line_loss_factor * loss_cost * delta_t * value(sum(R[l, k] * model[:I_sqr_k][l, k] * BASE_POWER for l in L, k in K))

        DSO_sub_op_costs = HOURS_PER_YEAR * f_s * sub_loss_factor * sub_op_cost * value(sum((model[:P_G][i]^2 + model[:Q_G][i]^2) * (delta_t * BASE_POWER)^2 for i in Ns))
    end

    kpis_header = (["DSO fixed line", "DSO fixed sub", "DSO loss", "DSO_sub_op_costs"],
    ["kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year"])

    kpis = sig_round([DSO_fixed_line DSO_fixed_sub DSO_loss DSO_sub_op_costs])
    
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end

function print_case_description(model, network_dict, peak_demand; Jabr=false, latex=false)
    solve_time       = JuMP.solve_time(model) 
    obj_value        = JuMP.objective_value(model)
    relative_gap     = JuMP.relative_gap(model)
    nb_int_vars      = JuMP.MOI.get(model, Gurobi.ModelAttribute("NumIntVars"))
    nb_bin_vars      = JuMP.MOI.get(model, Gurobi.ModelAttribute("NumBinVars"))
    nb_vars          = JuMP.MOI.get(model, Gurobi.ModelAttribute("NumVars"))
    nb_constraints   = JuMP.MOI.get(model, Gurobi.ModelAttribute("NumConstrs"))
    nb_Q_constraints = JuMP.MOI.get(model, Gurobi.ModelAttribute("NumQConstrs"))

    kpis_header = (
                [   "Model", "Objective", "Gap", "Solve Time", "Nb integer variables", "Nb binary variables", "Nb variables", "Nb constraints", "Nb Quadratic constraints", "Time periods", "Peak Demand"   ], 
                [   "", "kEUR/year", "%", "sec.", "" ,"","","","","", "MW"     ])

    params = sig_round(
                        [(Jabr ? "Jabr" : "BFM" ) obj_value relative_gap*100 solve_time nb_int_vars nb_bin_vars nb_vars nb_constraints nb_Q_constraints 1 peak_demand])

    if latex
        pretty_table(params, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(params, header=kpis_header)
    end
end 

# -- PRINT NETWORK RESULTS FUNCTION --
#
function print_network_results(model, network_dict; Jabr=false,latex=false)
    N, _, _, _, _, _, _ = network_dict[:bus]
    Ns, _, _ = network_dict[:sub_bus]
    Nu, _, _ , _ = network_dict[:load_bus]
    L, _, _, _, _, _, _ = network_dict[:line]
    K = network_dict[:conductor] 
    base_power = BASE_POWER
    HOURS_PER_YEAR = 8760 # h/year
    delta_t = 1 # 1h

    if Jabr
        nb_lines_built = value(sum(model[:x][l] for l in L))
    else 
        nb_lines_built = value(sum(model[:y_send][l] + model[:y_rec][l] for l in L))
    end

    nb_substations = value(sum(model[:beta][i] for i in Ns)) 

    substation_string = ["Substation "*string(i)*" capacity" for i in Ns]
    substation_capacities = [value(model[:S_G][i]) * base_power for i in Ns]

    cond_string = ["Nb lines cond "*string(i) for i in K]

    lines_cond = [[i for (i, a) in enumerate(value.(model[:alpha])[:, j]) if isapprox(a, 1, rtol=1e-2)] for j in K]
    
    kpis_header = ([["Nb lines built", "Nb substations"];cond_string;substation_string], [["-", "-"];fill("-", K);fill("MVA", Ns)])
    kpis = reshape(string.([[nb_lines_built, nb_substations];lines_cond;substation_capacities]),1,:)
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return
end


print_network_results (generic function with 1 method)

In [74]:
# -- COMPARE RESULTS OF THE TWO MODELS --
#
print_title("Jabr model:")
print_case_description(Jabr_model, network_dict, peak_demand; Jabr=true, latex=true)
print_cost_results(Jabr_model, network_dict, Jabr_obj_fct_dict; Jabr=true, latex=false)
print_network_results(Jabr_model, network_dict; Jabr=true, latex=false)

print_title("BFM model:")
print_case_description(BFM_model, network_dict, peak_demand; Jabr=false, latex=true)
print_cost_results(BFM_model, network_dict, BFM_obj_fct_dict; Jabr=false, latex=false)
print_network_results(BFM_model, network_dict; Jabr=false, latex=false)

Jabr model:
-----------
\begin{tabular}{rrrrrrrrrrr}
  \hline
  \textbf{Model} & \textbf{Objective} & \textbf{Gap} & \textbf{Solve Time} & \textbf{Nb integer variables} & \textbf{Nb binary variables} & \textbf{Nb variables} & \textbf{Nb constraints} & \textbf{Nb Quadratic constraints} & \textbf{Time periods} & \textbf{Peak Demand} \\
   & \texttt{kEUR/year} & \texttt{\%} & \texttt{sec.} &  &  &  &  &  &  & \texttt{MW} \\\hline
  Jabr & 1286.0 & 0 & 131.1 & 172 & 172 & 4888 & 2944 & 138 & 1 & 7.04 \\\hline
\end{tabular}
┌────────────────┬───────────────┬───────────┬──────────────────┐
│ DSO fixed line │ DSO fixed sub │  DSO loss │ DSO_sub_op_costs │
│      kEUR/year │     kEUR/year │ kEUR/year │        kEUR/year │
├────────────────┼───────────────┼───────────┼──────────────────┤
│          169.6 │        1100.0 │       0.0 │            16.72 │
└────────────────┴───────────────┴───────────┴──────────────────┘
┌────────────────┬────────────────┬────────────────────────────────────────────

In [24]:
# -- PRINTING THE RESULTING NETWORK --

function print_network_tikz(model, network_dict, x_scale, y_scale; Jabr=false, dir=pwd(), filename="graph", display=true, reshape=true)

    # ---------------- Preamble and postamble of the Latex document ----------------
    preamble = """\\documentclass{standalone}
        \\usepackage{tikz}
        \\usepackage{amsmath}
        \\usetikzlibrary{graphs, quotes, arrows.meta, positioning}
        \\usepackage{xcolor}
        \\definecolor{Ulg_blue}{RGB}{9, 111, 123}
        \\definecolor{Ulg_red}{RGB}{132, 38, 0}
        \\definecolor{Ulg_orange}{RGB}{223, 168, 120}
        \\begin{document}
        \\begin{tikzpicture}[
        every label/.style = {align=center, font=\\tiny, inner sep=2pt},
        every edge quotes/.style = {font=\\scriptsize, text=black, fill=white, inner sep=2pt}
        ]
        \\graph [no placement]
        {\n"""

    postamble = """};
        \\end{tikzpicture}
        \\end{document}"""

    style_sub = "rectangle, draw, fill=white, minimum size=1.7em, inner sep=1pt,  align=center, text width=0.6cm"
    style_load = "circle, draw, fill=white, minimum size=1.2em, inner sep=1pt, align=center, text width=0.6cm"

    # ---------------- Data required for the plot ----------------
    N, _, _, _, _, x_coordinates, y_coordinates = network_dict[:bus]
    Ns, _, _ = network_dict[:sub_bus]
    Nu, P_D, _ , _ = network_dict[:load_bus]
    L, line_ends, max_i, R, X, G, B = network_dict[:line]
    K = network_dict[:conductor]
    Ns_init = [1] 
    Ns_notinit = setdiff(Ns, Ns_init)
    base_power = BASE_POWER

    # Creating the .tex file
    file_path = joinpath(dir ,"$filename.tex")
    touch(file_path)

    # Writing the .tex file
    open(file_path, "w") do file
        write(file, preamble)

        for n in N
            x_coord = x_coordinates[n]
            y_coord = y_coordinates[n]
            if reshape
                if n in [1, 2]
                    x_coord += 0.5
                elseif n == 16
                    y_coord += 0.1
                elseif n == 20
                    y_coord -=  0.1
                elseif n in [3, 9]
                    if n == 9 
                        x_coord += 0.5  
                    end
                    y_coord -= 1.8
                elseif n in [4, 5]
                    if n == 4
                        x_coord += 0.8
                    else 
                        x_coord += 0.5
                    end
                    y_coord += 0.4
                end
            end
        x = x_scale * x_coord
        y = y_scale * y_coord

        ns = length(Ns)
        nu = n - ns
        if n <= ns
            sub_built = isapprox(value(model[:beta][n]), 1; rtol = 1e-4)
            P_gen = round(value(model[:P_G][n]); sigdigits = 2)
            col = P_gen >= 0 ? "\\color{Ulg_blue}" : "\\color{Ulg_red}"
            if sub_built
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$  \\vspace{0.1em}  \\scriptsize \$\\mathbf{$col $P_gen}\$}, $(style_sub), very thick];\n")
            else 
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$ \\vspace{0.1em}  \\scriptsize \$\\mathbf{$col $P_gen}\$}, $(style_sub)];\n")
            end
        else 
            P_cons = round(P_D[n]; sigdigits = 2)
            write(file,
            "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{U}_{$nu}\$  \\vspace{-0.2em} \\scriptsize \$ \\color{Ulg_red} \\mathbf{$P_cons} \$}, $(style_load)];\n")
        end
    end

    for l in L
        l_built = isapprox(value(sum(model[:alpha][l, :])), 1; rtol = 1e-4)      
        if l_built

            cond_idx = findfirst(x->isapprox(x, 1; rtol=1e-4), value.(model[:alpha][l, :]))

            if Jabr
                P_send = value(model[:P_ij][l]) * BASE_POWER
                P_rec  = value(model[:P_ji][l]) * BASE_POWER                
            
                if  P_send >= 0                   
                    i = line_ends[l][1] ; j = line_ends[l][2] 
                    p = P_send
                else 
                    i = line_ends[l][2] ; j = line_ends[l][1] 
                    p = abs(P_rec)
                end
    
            else 
                I_sqr    = value(model[:I_sqr][l])
                P_send   = value(model[:P_ij][l])
                P_rec    = P_send - (R[l, cond_idx] .* I_sqr^2) * BASE_POWER

                if  P_send >= 0                   
                    i = line_ends[l][1] ; j = line_ends[l][2] 
                    p = P_send
                else 
                    i = line_ends[l][2] ; j = line_ends[l][1] 
                    p = abs(P_rec)
                end
            end

            if cond_idx == 1
                write(file,
                "    $(i) ->[\"$(round(p, sigdigits=2))\"] $(j);\n")
            elseif cond_idx == 2
                write(file,
                "    $(i) ->[\"$(round(p, sigdigits=2))\", thick] $(j);\n")
            elseif cond_idx == 3
                write(file,
                "    $(i) ->[\"$(round(p, sigdigits=2))\", very thick] $(j);\n")
            elseif cond_idx == 4
                write(file,
                "    $(i) ->[\"$(round(p, sigdigits=2))\", ultra thick] $(j);\n")
            end
        else 
            i = line_ends[l][1] ; j = line_ends[l][2] 
            write(file,
            "    $(i) --[gray, dashed] $(j);\n")
        end       
    end  
        write(file, postamble)
end

    if display
        run(Cmd(`lualatex $filename.tex`, dir="$dir"))
        run(Cmd(`open $filename.pdf`, dir="$dir"))
    else
        run(Cmd(`lualatex $filename.tex`, dir="$dir"))
    end
end

print_network_tikz (generic function with 1 method)

In [25]:
thesis_dir = "/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures" 

Jabr_filename = "network_Jabr"
BFM_filename = "network_BFM"

print_network_tikz(BFM_model, network_dict, 5, 5; dir=thesis_dir, filename=BFM_filename, Jabr=false, display=true, reshape=true )

print_network_tikz(Jabr_model, network_dict, 5, 5; dir=thesis_dir, filename=Jabr_filename, Jabr=true, display=true, reshape=true )

This is LuaHBTeX, Version 1.15.0 (TeX Live 2022) 
 restricted system commands enabled.
(./network_BFM.tex
LaTeX2e <2021-11-15> patch level 1
 L3 programming layer <2022-02-24>
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2022/texmf-dist/tex/latex/tools/shellesc.sty)
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/ifluatex.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/iftex.sty))
(/usr/local/texlive/2022/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX 

Process(setenv(`open network_Jabr.pdf`; dir="/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures"), ProcessExited(0))

In [26]:
# -- FUNCTION THAT WRITES RESULTS IN XLSX FILES -- 

function write_results(XLSX_PATH::String, 
                      var_values::Dict, var_sets::Dict;
                      date_range = nothing
                      )

    for (key, value) in var_values
        if ndims(value) <= 2
            add_var_to_XLSX(XLSX_PATH, value, key, var_sets[key], date_range = date_range)
    
        elseif ndims(value) == 3
             X_i_ij = process_X_i_ij(value)
            add_var_to_XLSX(XLSX_PATH, X_i_ij, "X_i_ij", var_sets[key], date_range = date_range)
        end
    end
    
    return
end

write_results (generic function with 1 method)

In [27]:
# -- SAVING THE RESULTS OF THE MODELS IN EXCEL FILES --

include("post_process.jl")

XLSX_PATH_JABR = "output/Jabr_output.xlsx"
XLSX_PATH_BFM  = "output/BFM_output.xlsx"


write_results(XLSX_PATH_JABR, Jabr_var_values, Jabr_var_sets)
write_results(XLSX_PATH_BFM, BFM_var_values, BFM_var_sets)


In [ ]:
# -- OLD SHAPES OF THE NODES OF THE NETWORK --

house_nodeshape(x_i, y_i, s) = 
[
    (x_i + 0.5s * dx, y_i + 0.5s * dy) 
    for (dx, dy) in [(1, 1), (0, 1.6), (-1,1), (-1, -1), (1, -1), (1, 1)]
]

subs_nodeshape(x_i, y_i, s) = [
    (x_i + 0.9s * dx, y_i + 0.9s * dy) 
    for (dx, dy) in [(1, 1), (-1, 1), (-1, -1), (1, -1), (1,1)]
]

In [ ]:
# -- OLD FUNCTION THAT PRINTS THE NETWORK --

function print_network(nb_nodes::Int64, nb_sub_nodes::Int64, line_ends::Vector{Tuple{Int64, Int64}}, var_values; jabr::Bool=true)

    L  = 1:length(line_ends)
    N  = 1:nb_nodes 
    Ns = 1:nb_sub_nodes
    Nu = setdiff(N, Ns)

    alpha = sum(var_values["alpha"], dims=2)
    edge_label_dict = Dict{Tuple{Int64, Int64}, Float64}()
    
    if jabr
        P_ij = sum(var_values["P_ij"], dims=2) .* BASE_POWER
        P_ji = sum(var_values["P_ji"], dims=2) .* BASE_POWER
        line_power = [(P_ij[l] > 0 ? P_ij[l] : abs(P_ji[l])) for l in L]
    else
        P_ij = var_values["P_ij"] .* BASE_POWER
        I_sqr = var_values["I_sqr"]

        line_power = [(P_ij[l] > 0 ? P_ij[l] : abs(P_ij[l] - R[l]*I_sqr[l])) for l in L]
    end

    edge_width = Dict()
    g = SimpleDiGraph(nb_nodes)
    for l in L
        if P_ij[l] < 0
            add_edge!(g, line_ends[l][2], line_ends[l][1])
            key = (line_ends[l][2], line_ends[l][1])
        else
            add_edge!(g, line_ends[l][1], line_ends[l][2])
            key = line_ends[l]
        end
        if abs(line_power[l]) > 1e-9 # TO BE MODIFIED        
            edge_label_dict[key] = abs(round(line_power[l]; digits=3))
        end

        edge_width[key] = (isapprox(alpha[l], 1; rtol = 1e-4) ? 1 : 0.1)
    end
    colors = [colorant"lightseagreen", colorant"orange"]
  
    #=
    house_lables = Vector{String}(undef, length(Nu))
    subs_lables  = Vector{String}(undef, length(Ns))
    for n in Nu 
        P_G = round(var_values["P_G"][n]*BASE_POWER; digits=3)
        P_D = round(P_D[n]*BASE_POWER; digits=3)
        house_labels[n] = "+$(P_G) \n\n -$(P_D)"
    end 
    =#
    house_labels = ["+$(round(var_values["P_G"][n]*BASE_POWER, digits=3)) \n\n -$(round(P_D[n]*BASE_POWER, digits=3))" for n in Nu]
    subs_labels  = ["+$(round(var_values["P_G"][n]*BASE_POWER, digits=3))" for n in Ns]
    node_labels  = [subs_labels; house_labels]
    node_shapes  = [[subs_nodeshape for _ in Ns];[house_nodeshape for _ in Nu]]

    # See components.jl from Plots.jl to have all the argumentd
    graph  = graphplot( adjacency_matrix(g),
                        x               = [ [df_bus.x[n] + 0.5 for n in Ns];
                                            [df_bus.x[n] for n in Nu]],             # x-coordinate of the nodes
                        y               = df_bus.y,                                 # y-coordinate of the nodes
                        nodesize        = 0.1,
                        nodestrokewidth = 0,                                        # coutour line width of the node
                        edgestyle       = :solid,
                        nodealpha       = 1,                                        # transparency of node color
                        names           = node_labels,                              # node label
                        nodeshape       = node_shapes,                              # :circle, :ellipse, :hexagon
                        nodecolor       = colors[[[1 for _ in Ns];[2 for _ in Nu]]],
                        curves          = false,                                    # if an edge is curved or not
                        arrow           = Plots.arrow(:closed, 0.8, 0.8),                 # other choices : :open, :closed
                        ew              = edge_width,
                        shorten         = 0.05,
                        edgelabel       = edge_label_dict,
                        edgelabeloffset = 3,
                        axis_buffer     = 0.1,
                        fontsize        = 13,
                        size            = (1000, 3000)
                    )
    display(graph)

end

In [ ]:
# -- OLD PRINTING THE NETWORK --

pyplot()
print_network(length(N), length(Ns), line_ends, Jabr_var_values)
print_network(length(N), length(Ns), line_ends, BFM_var_values, jabr = false)

[comment]: <> (Section)
<hr style="color:#c6cde1;">
<p align="center">
    <b style="font-size:2vw; color:#c6cde1;">
    5. Functions that verify the results
    </b>
</p>
<hr style="color:#c6cde1;">

[comment]: <> (Description)
<p align="justify">
    In this section, one will be able to:
    <ul>
        <li style="margin-bottom:10px">Set the parameters that are required in the objective function;</li>
    </ul> 

In [ ]:
# -- JABR:  OPTIMALITY VERIFICATION --
@testset verbose = true "Checking optimality Jabr" begin
    P_G = Jabr_model[:P_G]
    Q_G = Jabr_model[:Q_G]
    S_G = Jabr_model[:S_G]
    X_i_ij  = Jabr_model[:X_i_ij]
    X_ij_re = Jabr_model[:X_ij_re]
    X_ij_im = Jabr_model[:X_ij_im]

    @testset "Conic constraints $i" for i in N
        @test S_G[i]^2 ≈ P_G[i]^2 + Q_G[i]^2
    end
    @testset "Rotated Conic constraints ($l, $k)" for l in L, k in K
        @test X_i_ij[l, k, line_ends[l][1]] * X_i_ij[l, k, line_ends[l][1]] ≈ X_ij_re[l, k]^2 + X_ij_im[l, k]^2
    end
end

In [ ]:
# -- BFM: OPTIMALITY VERIFICATION --

@testset verbose = true "Checking optimality BFM" begin
    P_G = BFM_model[:P_G]
    Q_G = BFM_model[:Q_G]
    S_G = BFM_model[:S_G]
    P_ij  = BFM_model[:P_ij]
    Q_ij  = BFM_model[:Q_ij]
    V_sqr = BFM_model[:V_sqr]
    I_sqr = BFM_model[:I_sqr]
    
    @testset "Conic constraints $i" for i in N
        @test S_G[i]^2 ≈ P_G[i]^2 + Q_G[i]^2
    end
    @testset "Rotated Conic constraints ($l, $k)" for l in L
        @test V_sqr[line_ends[l][1]] * I_sqr[l] ≈ P_ij[l]^2 + Q_ij[l, k]^2
    end
end